# Make Pokemon SV Datasets

In [2]:
import json
import pprint
from PIL import Image, ImageFilter
from PIL import ImageDraw

import glob
import re
import os
from loguru import logger
from tqdm import tqdm
import numpy as np

import cv2
import sys

## Setting param

In [21]:
capture_dir = "capture"
capture_video_dir = "video"
capture_image_dir = "image"

capture_video_path = capture_dir + "/" + capture_video_dir
capture_image_path = capture_dir + "/" + capture_image_dir

root_path = "/home"

diff_image_th = 1000
save_freq     = 4

datasets_dir  = "datasets"
datasets_ver  = "v0"
datasets_path = datasets_dir + "/" + datasets_ver

anotate_full = "datasets/v0/result.json"
anotate_full_repath = "datasets/v0/result_repath.json"

anotate_train_name = "pokemon_sv_train.json"
anotate_train_path = datasets_path + "/" + anotate_train_name
anotate_valid_name = "pokemon_sv_valid.json"
anotate_valid_path = datasets_path + "/" + anotate_valid_name



In [4]:
%cd $root_path

/home


In [5]:
!ls

Dockerfile  capture   docker-compose.yml     notebook	 test.json   utils
README.md   datasets  label_studio_exported  sample.txt  train.json


In [6]:
glob_path = capture_video_path + "/*.mp4"
video_list = glob.glob(glob_path, recursive=True)
pprint.pprint(video_list)

['capture/video/2022-11-19_11-55-09.mp4']


In [7]:
def analysis_video(video_path):
    
    video_name = video_path.split("/")[-1]
    video_single_path = capture_image_path + "/" + video_name
    logger.info("{:>20} : {}".format("video_single_path", video_single_path))
    os.makedirs(video_single_path, exist_ok=True)
       
    
    cap = cv2.VideoCapture(video_path)
    
    count = 0
    image_id = 1
    
    while True:
        ret, frame = cap.read()

        # 読み込み可能かどうか判定
        if ret:
            logger.info("========================")
            logger.info("{:>20} : {}".format("count", count))
            
            # 0番目は pre frameに登録のみで処理はskip
            if(count==0):
                pre_frame = frame
            else:
                # 0番目以降は処理
                
                # 差分を計算
                diff_image = np.sum(np.abs(pre_frame - frame))
                logger.info("{:>20} : {}".format("diff_image", diff_image))
                
                # 閾値以上なら処理する
                if(diff_image > diff_image_th):
                    # 一定間隔で画像を保存
                    if(image_id % save_freq == 0):
                        save_image_name = "{:09d}.jpg".format(image_id)
                        save_image_path = video_single_path + "/" + save_image_name
                        logger.info("{:>20} : {}".format("save_image_path", save_image_path))
                        cv2.imwrite(save_image_path, frame)
                                                          
                    image_id += 1
                pre_frame = frame
                
                
            count += 1
        else:
            logger.info("Video Fin ...")
            break
            
        

In [8]:
def video_section():
    for video_path in video_list:
        logger.info("{:>20} : {}".format("video_path", video_path))
        analysis_video(video_path)

In [9]:
#video_section()

In [23]:
with open(anotate_full, 'rt', encoding='UTF-8') as annotations:
    result_coco = json.load(annotations)

In [24]:
for i in range(len(result_coco["images"])):
    file_name = result_coco["images"][i]['file_name']    
    result_coco["images"][i]['file_name'] = file_name.split("/")[-1]

In [25]:
with open(anotate_full_repath, 'wt', encoding='UTF-8') as coco:
        json.dump(result_coco, coco, indent=2, sort_keys=True)

In [26]:
!python utils/cocosplit.py --having-annotations --multi-class -s 0.8 $anotate_full_repath $anotate_train_path $anotate_valid_path

Saved 87 entries in datasets/v0/pokemon_sv_train.json and 22 in datasets/v0/pokemon_sv_valid.json
